In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, PowerTransformer, RobustScaler

RSEED=826

In [2]:
# import data and immediate train test split
df = pd.read_csv('data/data.csv')
df_train, df_test = train_test_split(df, random_state=RSEED)

In [3]:
# df_train.info()

In [4]:
# df_train.head()

In [5]:
# drop columns with sensor angles as they are not useful for prediction
# as well as 'amf' columns as they hold same information as columns with almost identical name just in different format
for col in df_train.columns:
    if 'angle' in col or 'amf' in col[-3:]:
        df_train.drop([col], axis=1, inplace=True)
        df_test.drop([col], axis=1, inplace=True)

# drop CH4 columns as they include too many nan values
drop_cols = ['L3_CH4_CH4_column_volume_mixing_ratio_dry_air', 'L3_CH4_aerosol_height', 'L3_CH4_aerosol_optical_depth']
df_train.drop(drop_cols, axis=1, inplace=True)
df_test.drop(drop_cols, axis=1, inplace=True)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22917 entries, 3121 to 29745
Data columns (total 45 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Place_ID X Date                                  22917 non-null  object 
 1   Date                                             22917 non-null  object 
 2   Place_ID                                         22917 non-null  object 
 3   target                                           22917 non-null  float64
 4   target_min                                       22917 non-null  float64
 5   target_max                                       22917 non-null  float64
 6   target_variance                                  22917 non-null  float64
 7   target_count                                     22917 non-null  int64  
 8   precipitable_water_entire_atmosphere             22917 non-null  float64
 9   relative_humidity_2m_above_gro

In [6]:
# drop rows where target value is above 500 as this is already a very extreme value for pm2.5 concentration
df_train = df_train[df_train['target'] < 500]
df_test = df_test[df_test['target'] < 500]
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22912 entries, 3121 to 29745
Data columns (total 45 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Place_ID X Date                                  22912 non-null  object 
 1   Date                                             22912 non-null  object 
 2   Place_ID                                         22912 non-null  object 
 3   target                                           22912 non-null  float64
 4   target_min                                       22912 non-null  float64
 5   target_max                                       22912 non-null  float64
 6   target_variance                                  22912 non-null  float64
 7   target_count                                     22912 non-null  int64  
 8   precipitable_water_entire_atmosphere             22912 non-null  float64
 9   relative_humidity_2m_above_gro

In [7]:
# # visualize correlations between columns for each gas separately
# gases = ['NO2', 'O3', 'CO', 'HCHO', 'CLOUD', 'AER', 'SO2']

# fig, axes = plt.subplots(len(gases)+1, 1, figsize=(16, 100))
# axes = axes.flatten()

# for ax, gas in zip(axes, gases):
#     col_keeps = [col for col in df_train.columns if gas in col]
#     corr_gas = df_train[col_keeps + ['target']]
#     sns.heatmap(corr_gas.corr(), annot=True, ax=ax, vmin=-1, vmax=1)

# weather = ['humidity', 'wind', 'water', 'temperature']

# col_keeps = [col for col in df_train.columns for w in weather if w in col]
# corr_weather = df_train[col_keeps + ['target']]
# sns.heatmap(corr_weather.corr(), annot=True, ax=axes[-1], vmin=-1, vmax=1)

# plt.tight_layout(pad = 3)

In [8]:
# drop one of two columns which correlate strongly (>0.7) to prevent multicollinearity
use = ['target'] + list(df_train.columns[8:])
corr_df = df_train[use].corr().abs()

upper_tri = corr_df.where(np.triu(np.ones(corr_df.shape), k=1).astype(bool))

threshold = 0.7
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
print(f'Columns to drop [{len(to_drop)}]: {to_drop}\n')

df_train.drop(to_drop, axis=1, inplace=True)
df_test.drop(to_drop, axis=1, inplace=True)
df_train.info()

Columns to drop [17]: ['specific_humidity_2m_above_ground', 'temperature_2m_above_ground', 'L3_NO2_NO2_slant_column_number_density', 'L3_NO2_stratospheric_NO2_column_number_density', 'L3_NO2_tropospheric_NO2_column_number_density', 'L3_O3_cloud_fraction', 'L3_CO_H2O_column_number_density', 'L3_CO_sensor_altitude', 'L3_HCHO_cloud_fraction', 'L3_HCHO_tropospheric_HCHO_column_number_density', 'L3_CLOUD_cloud_base_pressure', 'L3_CLOUD_cloud_fraction', 'L3_CLOUD_cloud_top_height', 'L3_CLOUD_cloud_top_pressure', 'L3_AER_AI_absorbing_aerosol_index', 'L3_SO2_SO2_slant_column_number_density', 'L3_SO2_cloud_fraction']

<class 'pandas.core.frame.DataFrame'>
Index: 22912 entries, 3121 to 29745
Data columns (total 28 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Place_ID X Date                           22912 non-null  object 
 1   Date                                      22912 non-null  

In [9]:
# check for duplicates
df_train.duplicated().value_counts()

False    22912
Name: count, dtype: int64

In [10]:
# # visualize histograms of features to spot outliers or irregular distributions
# fix, axes = plt.subplots(7, 3, figsize=(20, 30))
# axes = axes.flatten()

# for ax, col in zip(axes, df_train.columns[8:]):
#     sns.histplot(df_train, x=col, ax=ax)

# plt.tight_layout(pad=3)

In [11]:
# # print descriptive metrics of features to spot outliers
# for col in ['target'] + list(df_train.columns[8:]):
#     print(f'''{col}
# min, mean, max:           {df_train[col].min().round(3)}   {df_train[col].mean().round(3)}   {df_train[col].max().round(3)}
# quantiles (25, 50, 75):   {df_train[col].quantile(0.25).round(3)}   {df_train[col].quantile(0.50).round(3)}   {df_train[col].quantile(0.75).round(3)}\n''')

# Baseline Model

In [12]:
# # scatterplot of feature with strongest correlation to target
# sns.scatterplot(df_train, x='L3_CO_CO_column_number_density', y='target')

In [13]:
def print_eval_metrics(y_test, y_pred, model_name):
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} Results")
    print(f"RMSE: {rmse:.3f}")
    print(f"R²:   {r2:.3f}")

In [14]:
# baseline model based on features associated with air pollution
def baseline_model(df, target_col, feature_cols=None, random_state=RSEED):
    if feature_cols == None:
        feature_cols = df.select_dtypes(include='number').columns.drop(target_col)

    X = df[feature_cols]
    y = df[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state) #, test_size=0.2

    # fill nan with median of train data
    X_train = X_train.fillna(X_train.median())
    X_test = X_test.fillna(X_train.median())

    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # evaluation
    print_eval_metrics(y_test,y_pred,"Linear Regression")

    return model, y_pred

print('-> based on features associated with air pollution')
cols = ["precipitable_water_entire_atmosphere", "relative_humidity_2m_above_ground", "u_component_of_wind_10m_above_ground", "v_component_of_wind_10m_above_ground"] # add clouds?
model, preds = baseline_model(df, target_col="target", feature_cols=cols, random_state=RSEED)

print('\n-> based on feature with strongest correlation to target')
cols = ['L3_CO_CO_column_number_density']
model, preds = baseline_model(df, target_col='target', feature_cols=cols, random_state=RSEED)

-> based on features associated with air pollution
Linear Regression Results
RMSE: 44.768
R²:   0.035

-> based on feature with strongest correlation to target
Linear Regression Results
RMSE: 42.855
R²:   0.116


In [15]:
# create list with all features that have weird bar at 0
peak_cols = [
    'L3_NO2_NO2_column_number_density',
    'L3_NO2_absorbing_aerosol_index',
    'L3_NO2_sensor_altitude',
    'L3_NO2_tropopause_pressure',
    'L3_O3_O3_column_number_density',
    'L3_O3_O3_effective_temperature',
    'L3_CO_CO_column_number_density',
    'L3_SO2_absorbing_aerosol_index'
    ]

# replace 0s with nan
df_train[peak_cols] = df_train[peak_cols].replace(0, np.nan)
df_test[peak_cols] = df_test[peak_cols].replace(0, np.nan)

In [16]:
# test_df = df_train.sort_values(by=['Place_ID', 'Date'])
# test_place = test_df[test_df['Place_ID'] == test_df['Place_ID'].unique()[9]]
# df_no_interp = pd.DataFrame({'Date': test_place['Date'], 'CO_density': test_place['L3_CO_CO_column_number_density']})

In [17]:
df_train = df_train.sort_values(by=['Place_ID', 'Date'])
nan_cols = df_train.columns[df_train.isna().any()].tolist()
df_train[nan_cols] = (df_train.groupby('Place_ID')[nan_cols]
                      .apply(lambda col: col.interpolate(method='linear').ffill().bfill())
                      .reset_index(level=0, drop=True))

df_test = df_test.sort_values(by=['Place_ID', 'Date'])
nan_cols = df_test.columns[df_test.isna().any()].tolist()
df_test[nan_cols] = (df_test.groupby('Place_ID')[nan_cols]
                     .apply(lambda col: col.interpolate(method='linear').ffill().bfill())
                     .reset_index(level=0, drop=True))

In [18]:
# test_df = df_train.sort_values(by=['Place_ID', 'Date'])
# test_place = test_df[test_df['Place_ID'] == test_df['Place_ID'].unique()[9]]
# df_interp = pd.DataFrame({'Date': test_place['Date'], 'CO_density': test_place['L3_CO_CO_column_number_density']})

# fig, axes = plt.subplots(1, 2, figsize=(20, 6))
# sns.barplot(df_no_interp, x='Date', y='CO_density', ax=axes[0])
# sns.barplot(df_interp, x='Date', y='CO_density', ax=axes[1])
# plt.tight_layout(pad=3)

# Pipeline

In [ ]:
# define train and test data and exclude columns of location and date as well as descriptive metrics of target
cols_places = ['Place_ID X Date', 'Date', 'Place_ID']
cols_target = ['target', 'target_min', 'target_max', 'target_variance', 'target_count']

X_train = df_train.drop(cols_places + cols_target, axis=1)
X_test = df_test.drop(cols_places + cols_target, axis=1)
y_train = df_train['target']
y_test = df_test['target']

<class 'pandas.core.frame.DataFrame'>
Index: 22912 entries, 1 to 30556
Data columns (total 20 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   precipitable_water_entire_atmosphere      22912 non-null  float64
 1   relative_humidity_2m_above_ground         22912 non-null  float64
 2   u_component_of_wind_10m_above_ground      22912 non-null  float64
 3   v_component_of_wind_10m_above_ground      22912 non-null  float64
 4   L3_NO2_NO2_column_number_density          22906 non-null  float64
 5   L3_NO2_absorbing_aerosol_index            22906 non-null  float64
 6   L3_NO2_cloud_fraction                     22912 non-null  float64
 7   L3_NO2_sensor_altitude                    22906 non-null  float64
 8   L3_NO2_tropopause_pressure                22906 non-null  float64
 9   L3_O3_O3_column_number_density            22912 non-null  float64
 10  L3_O3_O3_effective_temperature         

In [20]:
# select numerical cols
num_cols = X_train.select_dtypes('number').columns.tolist()

# identify skewed cols
skew_thresh = 0.5
skew_cols2 = X_train[num_cols].apply(lambda x: x.skew()).sort_values(ascending=False)
skew_cols = skew_cols2[abs(skew_cols2) > skew_thresh].index.tolist()
print(f"{len(skew_cols)} skewed cols: {skew_cols}")

# only select numerical cols which arent skewed
num_cols = list(set(num_cols) - set(skew_cols))
print(f"{len(num_cols)}     num cols:",num_cols)

14 skewed cols: ['L3_NO2_NO2_column_number_density', 'L3_CLOUD_cloud_optical_depth', 'L3_CLOUD_surface_albedo', 'L3_CO_CO_column_number_density', 'precipitable_water_entire_atmosphere', 'L3_CLOUD_cloud_base_height', 'L3_NO2_sensor_altitude', 'L3_AER_AI_sensor_altitude', 'L3_NO2_cloud_fraction', 'L3_CO_cloud_height', 'L3_NO2_absorbing_aerosol_index', 'relative_humidity_2m_above_ground', 'L3_O3_O3_effective_temperature', 'L3_SO2_SO2_column_number_density']
6     num cols: ['u_component_of_wind_10m_above_ground', 'L3_O3_O3_column_number_density', 'L3_SO2_absorbing_aerosol_index', 'L3_HCHO_HCHO_slant_column_number_density', 'L3_NO2_tropopause_pressure', 'v_component_of_wind_10m_above_ground']


In [21]:
# custom transformer - aka preprocessor encoder class for datetime
# ------------------------------------------
# split datetime into its components and add as feature
# capture cyclic nature of dayofweek with sin/cos
#   > important for LinRegression, NN
#   > not important (but not harmful either) for trees

class CyclicDatetimeEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, datetime_cols):
        self.datetime_cols = datetime_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ = X.copy()
        for col in self.datetime_cols:
            dt = X_[col]
            
            # simple calendar features
            X_[f"{col}_month"] = dt.dt.month
            X_[f"{col}_day"] = dt.dt.day
            X_[f"{col}_dayofweek"] = dt.dt.dayofweek
            X_[f"{col}_is_weekend"] = dt.dt.dayofweek.isin([5,6]).astype(int)

            # cyclic encoding for dayofweek
            X_[f"{col}_dow_sin"] = np.sin(2 * np.pi * dt.dt.dayofweek / 7)
            X_[f"{col}_dow_cos"] = np.cos(2 * np.pi * dt.dt.dayofweek / 7)

            X_.drop(columns=[col], inplace=True)

        # TransformerMixin:
        # - auto generate fit_transform
        # - to work seamless with pipelines
        # BaseEstimator:
        # - provides some standard methods: .get_params(), .set_params(), ...
        # - for Hyperparam tuning (eg GridSearchCV)
        
        return X_

In [22]:
prep_linreg = ColumnTransformer([
    ('skewed', Pipeline([
        # ('clip', FunctionTransformer(clip_outliers)),
        ('imputer', KNNImputer(n_neighbors=5)),
        ('power', PowerTransformer(method='yeo-johnson')),
        ('scaler', RobustScaler())
    ]), skew_cols),

    ('normal', Pipeline([
        # ('clip', FunctionTransformer(clip_outliers)),
        ('imputer', KNNImputer(n_neighbors=5)),
        ('scaler', RobustScaler())
    ]), num_cols),

    # ('datetime', CyclicDatetimeEncoder(dt_cols), dt_cols),
])

prep_tree = ColumnTransformer([
    ('num', KNNImputer(n_neighbors=5), num_cols + skew_cols),
    # ('datetime', CyclicDatetimeEncoder(dt_cols), dt_cols),
])

linreg = Pipeline([
    ('preprocessor', prep_linreg),
    ('model', LinearRegression())
])

elastic = Pipeline([
    ('preprocessor', prep_linreg),  # Use same preprocessing as LinearRegression
    ('model', ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=10000, random_state=RSEED))
])

ridge = Pipeline([
    ('preprocessor', prep_linreg),
    ('model', Ridge(alpha=1.0, random_state=RSEED))
])

lasso = Pipeline([
    ('preprocessor', prep_linreg),
    ('model', Lasso(alpha=1.0, max_iter=10000, random_state=RSEED))
])

tree_gbr = Pipeline([
    ('preprocessor', prep_tree),
    ('model', GradientBoostingRegressor(
            n_estimators=100,
            learning_rate=0.1,     # Shrinkage (lower = more conservative)
            max_depth=3,           # Shallow trees work best (3-5)
            min_samples_split=5,
            min_samples_leaf=2,
            subsample=0.8,         # Row sampling (0.5-1.0)
            random_state=RSEED
        ))
])

# tree_xgb = Pipeline([
#     ('preprocessor', prep_tree),
#     ('model', XGBRegressor(
#             n_estimators=100,
#             learning_rate=0.1,
#             max_depth=3,
#             min_child_weight=1,    # Min sum of weights in child
#             subsample=0.8,         # Row sampling
#             colsample_bytree=0.8,  # Column sampling
#             random_state=RSEED,
#             n_jobs=-1
#     ))
# ])

tree_rf = Pipeline([
    ('preprocessor', prep_tree),
    ('model', RandomForestRegressor(
    n_estimators=120,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt', #log2
    random_state=RSEED,
    n_jobs=-1))
])


# models = [linreg,ridge,elastic,lasso,tree_rf,tree_gbr,tree_xgb]
# # models = [linreg, tree_xgb]
# results = []
# for i, model in enumerate(models):
#     modelname = model.named_steps['model'].__class__.__name__
#     print(f"\nrunning pipeline #{i+1}/{len(models)} for {modelname}")

#     model.fit(X_train, y_train)    # applies all steps in pipeline to train-data
#     y_pred = model.predict(X_test) # apply same to test-data

#     # evaluation
#     res = print_eval_metrics(y_test,y_pred,modelname)
#     results.append(res)

In [23]:
# preprocess data outside of pipeline
X_train_prep = prep_tree.fit_transform(X_train)
X_test_prep  = prep_tree.transform(X_test)

# subsample to save time during GridSeachCV
X_train_sample = X_train.sample(frac=0.3, random_state=RSEED)
y_train_sample = y_train.loc[X_train_sample.index]

# preprocess sampled train data
X_train_sample_prep = prep_tree.fit_transform(X_train_sample)

In [24]:
def param_range(center, step_size, n_steps_each_side=2, min_val=1):
    start = max(center - n_steps_each_side * step_size, min_val)
    end = center + n_steps_each_side * step_size
    return list(range(start, end + step_size, step_size))

In [25]:
# hyperparameter tuning with SearchCV for RandomForestRegressor
param_grid_randCV = {
    'n_estimators': [100, 150, 200, 250, 300],
    'max_depth': [1, 5, 10, 15, 20],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [3, 5, 10, 15, 20],
    'max_features': ['sqrt', 'log2']
}

randCV = RandomizedSearchCV(RandomForestRegressor(random_state=RSEED), param_grid_randCV, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=RSEED)
randCV.fit(X_train_prep, y_train)
best_params = randCV.best_params_
print('Best params RandomSearchCV:', best_params)


param_grid_gridCV = {
    'n_estimators': param_range(best_params['n_estimators'], 20),
    'max_depth': param_range(best_params['max_depth'], 5),
    'min_samples_split': param_range(best_params['min_samples_split'], 1, min_val=2),
    'min_samples_leaf': param_range(best_params['min_samples_leaf'], 1),
    'max_features': [best_params['max_features']]
}

gridCV = GridSearchCV(RandomForestRegressor(random_state=RSEED), param_grid_gridCV, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)
gridCV.fit(X_train_sample_prep, y_train_sample)
best_params = gridCV.best_params_
print('Best params GridSearchCV:', best_params)


best_estimator = gridCV.best_estimator_

Best params RandomSearchCV: {'n_estimators': 250, 'min_samples_split': 5, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'max_depth': 20}
Best params GridSearchCV: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 210}


In [26]:
best_estimator.fit(X_train_prep, y_train)
y_pred = best_estimator.predict(X_test_prep)
print_eval_metrics(y_test, y_pred, 'RandomForestRegressor')

RandomForestRegressor Results
RMSE: 28.495
R²:   0.609


In [ ]:
# # plot feature importance (for tree models)
# rf = model.named_steps['model']
# feature_names = model.named_steps['preprocessor'].get_feature_names_out()

# importances = pd.Series(rf.feature_importances_, index=feature_names)
# importances.nlargest(30).plot(kind='barh', figsize=(8,6))
# plt.title("Top Feature Importances")
# plt.show()

In [28]:
# # visualize post-pipe
# fix, axes = plt.subplots(7, 3, figsize=(20, 30))
# axes = axes.flatten()

# for ax, col in zip(axes, X_train.columns):
#     sns.histplot(X_train, x=col, ax=ax)

# plt.tight_layout(pad=3)